# Chat-Bot включающий в себя обычную болталку и поиск по БАДам российского производства

## Загрузим необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

import re
import string
from pymorphy2 import MorphAnalyzer

import pickle

import annoy
from gensim.models import Word2Vec, FastText

import tensorflow as tf

import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from transformers import TFAutoModelForSequenceClassification

from telegram import Update
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [2]:
device = torch.device("cuda")
device

device(type='cuda')

In [3]:
morpher = MorphAnalyzer()
exclude = set(string.punctuation)

## Определим необходимые функции

In [4]:
# Функция предобработки текста для поиска в продуктах
def preprocess_txt_prod(line):
    # Удаляем знаки пунктуации
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    # Приводим слова к их нормальной форме
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    return spls

In [5]:
# Функция предобработки текста для обработки вопроса пользователя
def preprocess_txt(line):
    # Удаляем HTML теги
    spls = re.sub('<[^<]+?>', ' ', line)
    # Удаляем URLs
    spls = re.sub(r'http\S+', ' ', spls)
    # Убираем специальные символы: избавляемся от всего, что не является "словами"
    #print(spls)
    spls = re.sub('[^a-zA-Zа-яА-ЯёЁ0-9\s!?-]', ' ', spls)
    # Удаляем лишние пробелы
    spls = re.sub('\s+', ' ', spls)
    return spls

In [18]:
# Основная функция преобразования текста в вектор х100 для продуктов
def get_embedded_text(txt, idfs, midf, model):
    tokens = preprocess_txt_prod(txt)
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in tokens:
        if word in model.wv:
            vector_ft += model.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [7]:
# Функция поиска и вывода n (по умолчанию 5) релевантгых запросу продуктов
def search_product(request, idfs, midf, ft_index_shop, model, num_row=5):
    output = []
    
    vect_ft = get_embedded_text(request, idfs, midf, model)
    ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, num_row)
    for item in ft_index_shop_val:
        name, link = index_map_shop[item]
        output.append([name, link])
    return output

In [8]:
# Функция генерации ответов для общей болталки
def respond_to_dialog(texts, model, tokenizer):
    prefix = '\nq:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nq:' if i % 2 == 1 else '\na:'
    #print(prefix)
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [24]:
# Функция запускающая и поддерживающая диалог с пользователем
def start_dialog(tokenizer, model, tokenizer2, model2, model3, idfs, midf, ft_index_shop):
    seed = input('Начните диалог с ботом любой фразой\n Для завершения диалога введите: Stop\n')
    history = [seed]
    is_continue_dilog = True
    
    while is_continue_dilog:
        input_txt = preprocess_txt(seed)
        pred, label = get_prediction(input_txt, tokenizer2, model2)
        #print(label)
    
        # ПРОДУКТЫ
        if label == 0:
            prod_list = search_product(input_txt, idfs, midf, ft_index_shop, model3)
            for item in prod_list:
                prod_name, prod_link = item[0], item[1]
                print("Наименование продукта: {}\nСсылка на продукт: {}".format(prod_name, prod_link))
            seed = input('\n')
            
        # QA
        if label == 1:
            if seed == 'Stop':
                break
           
            output = respond_to_dialog([seed], model, tokenizer)
            seed = input(output + '\n')
      
            history.append(output)
            history.append(seed)

In [10]:
# Функция определения класса болталки: общая или продуктовая
def get_prediction(sentence, tokenizer, model):
    tokenized_sent = tokenizer(sentence, padding='max_length', truncation=True, return_tensors='tf')
    pred = model(tokenized_sent).logits.numpy()
    return pred, pred.argmax()

Классификатор интентов

In [11]:
classification_tokenizer = AutoTokenizer.from_pretrained("classifier_tk")
classification_model = TFAutoModelForSequenceClassification.from_pretrained("classifier_model")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at classifier_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


Модель обычной болталки

In [12]:
common_talker_tokenizer = AutoTokenizer.from_pretrained("final_ct_gpt_tk")
common_talker_model = AutoModelForCausalLM.from_pretrained("final_ct_gpt_model").to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Модель для продуктовой болталки

In [13]:
# Загружаем предобученую модель FastText
products_model = FastText.load("ft_model")

# Загружаем предобученный TFIDF векторайзер 
with open('tfidf.pkl', 'rb') as f:
    tfidf_vect = pickle.load(f)

idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
midf = np.mean(tfidf_vect.idf_)

# Загружаем продуктовые индексы
ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('bads_shop.ann') 
index_map_shop = pd.read_pickle("index_bads_shop.pkl")

#### Проверка через внутрений диалог

In [25]:
start_dialog(common_talker_tokenizer, common_talker_model, classification_tokenizer, classification_model, products_model, idfs, midf, ft_index_shop)

Начните диалог с ботом любой фразой
 Для завершения диалога введите: Stop
 Привет!
привет!!! ты кто??? давай знакомиться!!!
 Я Влад, А тебя как зовут?
аня!! ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы
 Что делаеш?
делаю все что хочу!! ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы ы
 Я сегодня нашел миллион рублей что мне делать?
забудь!!! ничего не теряй и живи дальше!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 Я ищу витамины
витамины группы в1 а так же комплексные минеральные добавки для роста волос и кожи лица u-beauty - это натуральные продукты которые помогают организму усваивать все необходимые человеку вещества без вреда для здоровья! u-beauty содержит большое количество аминокислот способствующих росту волос и укреплению иммунной системы кроме того он богат м

Наименование продукта: Эльзам, Кордицепс (биоактивный концентрат), 10х10 мл
Ссылка на продукт: https://itab.pro/products/elzam-korditseps-bioaktivnyy-kontsentrat-10h10-ml-1884/
Наименование продукта: FINE, Хлорелла, таблетки, 1500 шт.
Ссылка на продукт: https://itab.pro/products/fine-hlorella-tabletki-1500-sht-2076/
Наименование продукта: Maxler, Витамин D3+K2, мягкие капсулы, 90 шт.
Ссылка на продукт: https://itab.pro/products/maxler-vitamin-d3k2-myagkie-kapsuly-90-sht-415/
Наименование продукта: Trace Minerals, Ионные витамины D3+K2, жидкость, 59 мл
Ссылка на продукт: https://itab.pro/products/trace-minerals-ionnye-vitaminy-d3k2-zhidkost-59-ml-1815/
Наименование продукта: FINE, Витамин С + D3, капсулы, 60 шт.
Ссылка на продукт: https://itab.pro/products/fine-vitamin-s-d3-kapsuly-60-sht-2090/



 витаминный комплекс


Наименование продукта: Trace Minerals, Комплекс «Trace Mineral Drops» (для поддержания здоровья организма), жидкость, 15 мл
Ссылка на продукт: https://itab.pro/products/trace-minerals-kompleks-trace-mineral-drops-dlya-podderzhaniya-zdorovya-organizma-zhidkost-15-ml-1812/
Наименование продукта: Trace Minerals, Комплекс «Trace Mineral Drops» (для поддержания здоровья организма), жидкость, 237 мл
Ссылка на продукт: https://itab.pro/products/trace-minerals-kompleks-trace-mineral-drops-dlya-podderzhaniya-zdorovya-organizma-zhidkost-237-ml-1813/
Наименование продукта: Natures Plus, Animal Parade Source of Life, Детский витаминно-минеральный комплекс, (вишня, апельсин, виноград), жевательные таблетки, 90 шт.
Ссылка на продукт: https://itab.pro/products/natures-plus-animal-parade-source-of-life-detskiy-vitaminno-mineralnyy-kompleks-vishnya-apelsin-vinograd-zhevatelnye-tabletki-90-sht-1835/
Наименование продукта: Trace Minerals, Комплекс «Kid's Multi» (комплекс витаминов и минералов), жидкость,


 Омега-3
омега-3 гидроксид натрия гидроксид кальция гидроксид железа гидроксид меди гидроксид алюминия гидроксид магния гидроксид редкоземельных металлов гидроксид кремния гидроксид свинца гидроксид железа гидроксид меди гидроксид железа гидроксид углерода гидроксид меди гидроксид меди гидроксид водорода гидроксид меди гидроксид железа гидроксид азота гидроксид меди гидроксид железа гидроксид железа гидроксид железа гидроксид железа гидроксид железа гидроксид меди гидроксид железа гидроксид железа гид
 Omega-3


Наименование продукта: VIRIDI LABORATORIUM, Omega-3 Plus, капсулы, 30 шт.
Ссылка на продукт: https://itab.pro/products/viridi-laboratorium-omega-3-plus-kapsuly-30-sht-1803/
Наименование продукта: VIRIDI LABORATORIUM, Omega-3 Plus, капсулы, 180 шт.
Ссылка на продукт: https://itab.pro/products/viridi-laboratorium-omega-3-plus-kapsuly-180-sht-1802/
Наименование продукта: Avicenna, D Max 5, Витамин Д3 (5000 МЕ), капсулы, 60 шт.
Ссылка на продукт: https://itab.pro/products/avicenna-d-max-5-vitamin-d3-5000-me-kapsuly-60-sht-1977/
Наименование продукта: Avicenna, D Max 10, Витамин Д3 (10000 МЕ), капсулы, 60 шт.
Ссылка на продукт: https://itab.pro/products/avicenna-d-max-10-vitamin-d3-10000-me-kapsuly-60-sht-1978/
Наименование продукта: Avicenna, D Max 2, Витамин Д3 (2000 МЕ), капсулы, 60 шт.
Ссылка на продукт: https://itab.pro/products/avicenna-d-max-2-vitamin-d3-2000-me-kapsuly-60-sht-1976/



 Stop


## Создание Chat-Bot

In [26]:
# заменить на свой токен
updater = Updater("6345152081:AAE0bNMtN_AigXTorBgqApF4hoqgmmP1RAg", use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

In [ ]:
def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Привет!\nЯ чат-бот поиска биоактивных добавок российского производства!\nЯ так же люблю и просто поболтать!\nЧем могу помочь?')

def textMessage(update, context):
    
    input_txt = preprocess_txt(update.message.text)
    pred, label = get_prediction(input_txt, classification_tokenizer, classification_model)
    
    # ПРОДУКТЫ
    if label == 0:
        prod_list = search_product(input_txt, idfs, midf, ft_index_shop, products_model)
        for item in prod_list:
            prod_name, prod_link = item[0], item[1]
            context.bot.send_message(chat_id=update.message.chat_id, text="Наименование продукта: {}\nСсылка на продукт: {}".format(prod_name, prod_link))
        return
    
    # QA
    if label == 1:
        answer = respond_to_dialog([update.message.text], common_talker_model, common_talker_tokenizer)
        context.bot.send_message(chat_id=update.message.chat_id, text=answer)
        return
    

        
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

Exiting immediately!
